In [ ]:

from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
# Ensure the below library is installed on your cluster
import reverse_geocoder as rg
from datetime import date, timedelta

In [0]:
import json

# Get base parameters
dbutils.widgets.text("bronze_params", "")
dbutils.widgets.text("silver_params", "")

bronze_params = dbutils.widgets.get("bronze_params")
silver_params = dbutils.widgets.get("silver_params")

# Debug: Print the raw input values for troubleshooting
print(f"Raw bronze_params: {bronze_params}")
print(f"Raw silver_params: {silver_params}")

# Parse the JSON string
bronze_data = json.loads(bronze_params)

# Access individual variables
start_date = bronze_data.get("start_date", "")
end_date = bronze_data.get("end_date", "")
silver_adls = bronze_data.get("silver_adls", "")
gold_adls = bronze_data.get("gold_adls", "")
silver_data = silver_params

# Debug: Print the extracted values for verification
print(f"Start Date: {start_date}, End Date: {end_date}")
print(f"Silver ADLS Path: {silver_adls}, Gold ADLS Path: {gold_adls}")


' Data Factory\n\nimport json\n\n# Get base parameters\ndbutils.widgets.text("bronze_params", "")\ndbutils.widgets.text("silver_params", "")\n\nbronze_params = dbutils.widgets.get("bronze_params")\nsilver_params = dbutils.widgets.get("silver_params")\n\n# Debug: Print the raw input values for troubleshooting\nprint(f"Raw bronze_params: {bronze_params}")\nprint(f"Raw silver_params: {silver_params}")\n\n# Parse the JSON string\nbronze_data = json.loads(bronze_params)\n\n# Access individual variables\nstart_date = bronze_data.get("start_date", "")\nend_date = bronze_data.get("end_date", "")\nsilver_adls = bronze_data.get("silver_adls", "")\ngold_adls = bronze_data.get("gold_adls", "")\nsilver_data = silver_params\n\n# Debug: Print the extracted values for verification\nprint(f"Start Date: {start_date}, End Date: {end_date}")\nprint(f"Silver ADLS Path: {silver_adls}, Gold ADLS Path: {gold_adls}")\n'

In [0]:
df = spark.read.parquet(silver_data).filter(col('time') > start_date)

In [0]:

def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    try:
        coordinates = (float(lat), float(lon))
        result = rg.search(coordinates)[0].get('cc')
        print(f"Processed coordinates: {coordinates} -> {result}")
        return result
    except Exception as e:
        print(f"Error processing coordinates: {lat}, {lon} -> {str(e)}")
        return None

In [0]:
# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

In [0]:
# adding country_code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

In [0]:
# adding significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )
     


In [0]:

df_with_location_sig_class.printSchema()


root
 |-- id: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- elevation: double (nullable = true)
 |-- title: string (nullable = true)
 |-- place_description: string (nullable = true)
 |-- sig: long (nullable = true)
 |-- mag: double (nullable = true)
 |-- magType: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- updated: timestamp (nullable = true)
 |-- country_code: string (nullable = true)
 |-- sig_class: string (nullable = false)



In [0]:
     
# Save the transformed DataFrame to the gold container
gold_output_path = f"{gold_adls}earthquake_events_gold/"

In [0]:

# Append DataFrame to Silver container in Parquet format
df_with_location_sig_class.write.mode('append').parquet(gold_output_path)